<a href="https://colab.research.google.com/github/larsmoan/COMP3710/blob/main/assignement_2/cnn/cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from sklearn.datasets import fetch_lfw_people
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import numpy as np
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
from PIL import Image


def show_pic_tensor(img_tensor):
  image_np = img_tensor.numpy()
  # Display the grayscale image in black and white
  plt.imshow(image_np, cmap='gray', vmin=0, vmax=1)  # Use the 'gray' colormap
  plt.axis('off')  # Turn off axis labels and ticks
  plt.show()

In [2]:
# Fetch the LFW dataset
lfw_people = fetch_lfw_people(min_faces_per_person=70, resize=0.4)
n_classes = lfw_people.target_names.shape[0]

# Convert images to grayscale
X_gray = [Image.fromarray(image).convert('L') for image in lfw_people.images]

# Convert to NumPy array and reshape to (num_samples, 1, height, width)
X_gray = np.array([np.array(image) for image in X_gray])
X_gray = X_gray.reshape(-1, 1, 50, 37)

# Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_gray, lfw_people.target, test_size=0.2, random_state=42)

# Convert the NumPy arrays to PyTorch tensors
X_train = torch.from_numpy(X_train).float()
X_test = torch.from_numpy(X_test).float()
y_train = torch.from_numpy(y_train)
y_test = torch.from_numpy(y_test)

# Create TensorDatasets for train and test data
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

# Define batch size for the data loaders
batch_size = 6

# Create DataLoader instances for train and test data
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [3]:
# Define the CNN model
class CNNClassifier(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.network = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),  # Changed input channels to 1
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),  # Added another convolutional layer
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Flatten(),
            nn.Linear(64 * 12 * 9, 128),  # Adjusted input size
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        return self.network(x)

In [4]:
#Training

num_epochs = 300
learning_rate = 0.001

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNNClassifier(num_classes=n_classes).to(device)

# Define the loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(train_loader)}")


Epoch 1/300, Loss: 1.8155895017607266
Epoch 2/300, Loss: 1.766697135082511
Epoch 3/300, Loss: 1.7278085208909457
Epoch 4/300, Loss: 1.7357590555451636
Epoch 5/300, Loss: 1.7332595909057662
Epoch 6/300, Loss: 1.7262373067611871
Epoch 7/300, Loss: 1.7229205411533977
Epoch 8/300, Loss: 1.7176638231721035
Epoch 9/300, Loss: 1.7178061202515003
Epoch 10/300, Loss: 1.7131547851617945
Epoch 11/300, Loss: 1.7163205094808756
Epoch 12/300, Loss: 1.7153808592363846
Epoch 13/300, Loss: 1.7137964575789695
Epoch 14/300, Loss: 1.7314772096484206
Epoch 15/300, Loss: 1.71254608693511
Epoch 16/300, Loss: 1.707419627627661
Epoch 17/300, Loss: 1.707587756736334
Epoch 18/300, Loss: 1.7111458251642626
Epoch 19/300, Loss: 1.7064289575399354
Epoch 20/300, Loss: 1.6982786762853002
Epoch 21/300, Loss: 1.7078357387420744
Epoch 22/300, Loss: 1.7017507754092993
Epoch 23/300, Loss: 1.7009652794100518
Epoch 24/300, Loss: 1.6993994380152502
Epoch 25/300, Loss: 1.7039001029591228
Epoch 26/300, Loss: 1.7025860721288726


In [6]:
# Test loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy}%")

Test Accuracy: 45.736434108527135%
